# Ejercicio 1: Algoritmo *Naive Bayes*

Este algoritmo se caracteriza por ser extremadamente rápido de computar, y sus aplicaciones se extienden a numerosos campos de la inteligencia artificial. Sin embargo, cuenta con un gran inconveniente (presente en la hipótesis del teorema de Bayes), y es que las *features* se asumen independientes, lo cual es muy poco frecuente en la vida real (aunque puede conseguirse mediante técnicas como *PCA*).

El teorema de Bayes nos da una forma de calcular probabilidades de eventos *a posteriori* conociendo la información *a priori*:

$$ P(c|x)=\frac{P(x|c) P(c)}{P(x)} $$

En términos relacionados a nuestro contexto, podría leerse como sigue: *la probabilidad de una determinada clase dada una serie de features, es igual al producto de la probabilidad de dichas features dada una clase, por la probabilidad de dicha clase y el inverso de las features*.

Nuestro objetivo en este ejercicio será replicar el algoritmo de *Naive Bayes* con una implementación que mimetice a la de la famosa librería *scikit-learn*. Para ello, dado un cierto *array* de *features* y otro *array* con las etiquetas a predecir, debemos de calcular:

* Las probabilidades *a priori* en un método llamado `calc_prior` que se almacenen en una variable interna de la clase llamada `prior`.
* Las estadísticas de cada grupo de etiquetas (media y varianza) en un método llamado `calc_statistics` que se almacenen en una variable interna de la clase llamada `mean` y `var` respectivamente.
* La probabilidad de cada grupo. Este apartado depende de la asunción que hagamos sobre la distribución de nuestros datos; en este caso, para no distraer la atención del ejercicio, lo damos hecho (método `gaussian_density`) y asumiremos que nuestras *features* son todas continuas y se distribuyen de forma normal.
* La probabilidad *a posteriori*. Esto también lo damos hecho por simplicidad.
* Un método que nos permita sacar estos cálculos de unos determinados datos de entrenamiento. Esto también lo damos hecho por simplicidad.
* Un método que nos permita hacer predicciones en datos en los que el modelo no ha sido entrenado. Esto también lo damos hecho por simplicidad.

Partimos del conjunto de datos `iris`:

In [1]:
# Requerimientos
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
# Cargar datos
features, target = load_iris(return_X_y = True, as_frame=True)

Procedamos ahora a la implementación de los dos pequeños métodos de nuestro algoritmo:

In [2]:
class NaiveBayesClassifier():
  
    def calc_prior(self, features, target):
        '''
        Método para calcular probabilidades a priori
        '''
        # ESCRIBE AQUÍ LAS PROBABILIDADES A PRIORI POR ETIQUETA
        self.prior = None 
        return self.prior


    def calc_statistics(self, features, target):
        '''
        Calculamos media y varianza de cada columna
        '''
        # ESCRIBE AQUÍ LAS MÉTRICAS POR ETIQUETA
        self.mean = None 
        self.var = None 
              
        return self.mean, self.var


    def gaussian_density(self, class_idx, x):     
        '''
        calculate probability from gaussian density function (normally distributed)
        we will assume that probability of specific target value given specific class is normally distributed
        
        probability density function derived from wikipedia:
        (1/√2pi*σ) * exp((-1/2)*((x-μ)^2)/(2*σ²)), where μ is mean, σ² is variance, σ is quare root of variance (standard deviation)
        '''
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob


    def calc_posterior(self, x):
        posteriors = []

        # Calculamos la probabilidad posterior de cada clase
        for i in range(self.count):
            prior = np.log(self.prior[i])
            conditional = np.sum(np.log(self.gaussian_density(i, x)))
            posterior = prior + conditional
            posteriors.append(posterior)
        # Devuelve clase con mayor probabilidad a posteriori
        return self.classes[np.argmax(posteriors)]


    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)


    def predict(self, features):
        y_pred = np.array([self.calc_posterior(f) for f in features.to_numpy()])
        return y_pred

A continuación, en esta breve pieza de código podremos brevemente adelantarnos a la parte de modelización del temario para hacer predicciones con este modelo:

In [4]:
# Método para estratificar nuestros datos
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# Dividimos los datos de forma que haya la misma proporción de etiquetas en cada partición
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=.2, stratify=target)
# Instanciamos la clase anterior, entrenamos en una partición y predecimos en la otra
NBC = NaiveBayesClassifier()
NBC.fit(train_features, train_target)
predicciones = NBC.predict(test_features)
# Vemos cuál ha sido el rendimiento del modelo
print(classification_report(test_target, predicciones))

TypeError: 'NoneType' object is not subscriptable

# Ejercicio 2: Máxima verosimilitud

Realice un estudio similar al de los apuntes para estimar el parámetro $\lambda$ de una distribución Poisson mediante la máxima verosimilitud. La distribución de Poisson se encuentra en multitud de fenómenos naturales y es por ello fruto de modelización en múltiples experimentos, por lo que su conocimiento es fundamental.

Para ello, siga los pasos:

* Dado que el dominio de una distribución Poisson son los números enteros no negativos, considere una serie de $n$ experimentos independientes e idénticamente distribuídos donde en cada uno de ellos resulta tener lugar el suceso $\{n_i\}_{i=0}^n$. 
* Dado que son sucesos independientes, la probabilidad conjunta se expresa como producto de probabilidades marginales.
* Aplique el logaritmo y simplifique esta expresión.
* Utilice derivadas e iguale a cero la expresión para despejar $\hat{\lambda}$.

Recuerde que la función masa de probabilidad de una distribución $X\sim \mathcal{P}(\lambda)$ es $P[X=k]=\frac{\lambda^k \cdot e^{-\lambda}}{k!}$.


# Ejercicio 3: Valores atípicos

Construya una función que, dado un conjunto de variables continuas en forma de columnas en un objeto `pd.DataFrame`, realice los siguientes pasos para cada una de ellas:

* Calcule los percentiles 75 y 25.
* Calcule el rango intercuartílico.
* Calcule el intervalo $ \left[Q_1-1.5\cdot\text{IQR}, Q_3+1.5\cdot\text{IQR}\right] $.
* Localice los valores que no estén en este intervalo por ser **inferiores** y reemplácelos por $Q_1-1.5\cdot\text{IQR}$.
* Localice los valores que no estén en este intervalo por ser **superiores** y reemplácelos por $Q_3+1.5\cdot\text{IQR}$.

Se valorará adicionalmente a aquellos alumnos que sean capaces de añadir además que se pueda hacer un estudio de valores atípicos agrupando por variables con alta correlación con la variable respuesta.

Aplique lo anterior para analizar las variables continuas del siguiente *dataset*:

```python
# Requerimientos
import os
import pandas as pd
# Creamos carpeta
!mkdir /content/air_quality_dataset
# Movemos directorio activo
%cd /content/air_quality_dataset
# Descargamos fichero comprimido
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00501/PRSA2017_Data_20130301-20170228.zip
# Descargamos el fichero que contiene los datos a nuestro directorio activo
!unzip PRSA2017_Data_20130301-20170228.zip
# Nos movemos a la carpeta que contenía el zip
%cd PRSA_Data_20130301-20170228
# Leemos todos los archivos en una línea
df = pd.concat([pd.read_csv(elem) for elem in os.listdir()]).reset_index(drop=True)
```

# Ejercicio 4: Correlación

Implemente un método que tenga los siguientes pasos:

* Cree una matriz de correlación de las vriables continuas de un *dataset* y lo compare a la variable respuesta.
* Si hay dos variables altamente correlacionadas, elimine del *dataset* la variable que menos correlacionada esté con la variable respuesta.

Para ello, come como referencia el siguiente *dataset*:

```python
# Dependencias
import pandas as pd
# Descargamos el fichero que contiene los datos a nuestro directorio activo
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00514/Bias_correction_ucl.csv
# Leemos fichero
df = pd.read_csv('Bias_correction_ucl.csv')
```

Tome como variable respuesta `Next_Tmax`.